In [1]:
import pandas as pd
import requests
import json
import numpy as np
from collections import Counter

def searching(
    query_string,
    broad='no',
    has_ngrams='true',
    does_date_matter = 'yes',
    from_date='2010-01-01',
    to_date='2022-12-12'):

    if broad == 'no':
        if does_date_matter == 'yes':
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams},from_publication_date:{from_date},to_publication_date:{to_date}&page=1&per-page=20'
            ).json()
        else:
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams}&page=1&per-page=20'
            ).json()

    elif broad == 'yes':
        institution = requests.get(
            f'https://api.openalex.org/works?search={query_string}&filter=has_ngrams:{has_ngrams}&page=1&per-page=20'
        ).json()
    
    a = []
    for x in range(30):
        try:
            y = institution['results'][x]['ngrams_url']
            if x < 5:
                print(institution['results'][x]['title'])
                print(institution['results'][x]['authorships'][0]['author']['display_name'])
                print(institution['results'][x]['doi'])
                print(institution['results'][x]['relevance_score'])
                print(institution['results'][x]['publication_date'])
                print(y,'\n')  
            z = requests.get(y).json()
            a.append(z)
        except:
            pass

    return a

# _ngrams = searching('yukon hydrology',broad='no')
# _ngrams = searching('nitrogen pollution',broad='no')
_ngrams = searching('columbia river',broad='no')

# for idx,i in enumerate(_ngrams):
#     if idx == 0:
#         j = i['ngrams']
#         k = pd.DataFrame.from_dict(j)
#     else:
#         j = i['ngrams']
#         l = pd.DataFrame.from_dict(j)
#         k = pd.concat([k,l])

Implications of 21st century climate change for the hydrology of Washington State
Marketa M. Elsner
https://doi.org/10.1007/s10584-010-9855-0
675.75635
2010-05-04
https://api.openalex.org/works/W2103860021/ngrams 

Linkages between hydrological drought, climate indices and human activities: a case study in the Columbia River basin
Shengzhi Huang
https://doi.org/10.1002/joc.4344
639.70215
2016-01-01
https://api.openalex.org/works/W1919196885/ngrams 

Flame Retardant Transfers from U.S. Households (Dust and Laundry Wastewater) to the Aquatic Environment
Erika Schreder
https://doi.org/10.1021/es502227h
632.44446
2014-09-17
https://api.openalex.org/works/W2329876460/ngrams 

Determining indicator taxa across spatial and seasonal gradients in the Columbia River coastal margin
Caroline S. Fortunato
https://doi.org/10.1038/ismej.2013.79
572.9528
2013-05-30
https://api.openalex.org/works/W1990241939/ngrams 

Impact of reservoirs on river discharge and irrigation water supply during the 20th ce

In [2]:
ngram = []

for i in _ngrams:
    for j in i['ngrams']:
        ngram.append((j['ngram'].lower(),j['ngram_count']))

print(len(ngram))

b = Counter()
for ng,co in ngram:
    b[ng] += co

c = [[x,y] for x,y in b.items()]

badwords = ['add words to filter as necessary',' .$', 'et al','et$',]

df = pd.DataFrame.from_records(c,columns=['ngram','ngram_count'])
df['ngram_tokens'] = df['ngram'].str.split().str.len() #https://stackoverflow.com/questions/37483470/how-to-calculate-number-of-words-in-a-string-in-dataframe
df1 = df.dropna(axis=0)
df2 = df1.loc[(df1['ngram_tokens'] > 1) & (df1['ngram_tokens'] < 5)]
df3 = df2.sort_values(by=['ngram_count','ngram_tokens'],ascending=False)

for i in badwords:
    df3 = df3[~df3['ngram'].str.contains(i)]
df3.head(n=30)

130536


,ngram,ngram_count,ngram_tokens
7764,columbia river,546,2
6232,river basin,159,2
6165,climate change,138,2
30903,chinook salmon,134,2
4189,columbia river basin,127,3
13348,snake river,110,2
96092,predation rate,102,2
13359,upper columbia,98,2
31670,pit tag,78,2
65154,m 3 s à1,74,4


In [3]:
queries = {}

for i in range(5):
   queries[f'0{i}'] = df3['ngram'].iloc[i]

# c = 0
# for i in queries:
#     c+=1

# print(c)

string = []
for idx,i in enumerate(queries):
    if idx == 0:
        string.append(
            f"{queries[i]}"
            f""
            )
    else:
        string.append(
            f"|"
            f"{queries[i]}"
            f""
            )
x = ''.join(string)
x

'columbia river|river basin|climate change|chinook salmon|columbia river basin'

In [4]:
# search = requests.get(
#     f"https://api.openalex.org/works?"
#     f"filter="
#     f"title.search:{x}|"

#     f"&sort=relevance_score:desc&page=1&per-page=20"
# ).json()

# resultlist = []

# for x in range(10):
#     result = {}
#     for y in search['results'][x].keys():
#         result[y] = search['results'][x][y]
#     resultlist.append(result)

# x = [i['title'] for i in resultlist]
# y = [i['authorships'][0]['author']['display_name'] for i in resultlist]
# z = [i['doi'] for i in resultlist]
# zz = [i['relevance_score'] for i in resultlist]

# for idx, i in enumerate(x):
#     print(i)
#     print(y[idx])
#     print(z[idx])
#     print(zz[idx])
#     print('\n')